# L5: Tokenization

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [1]:
import sys
#!{sys.executable} -m pip install tiktoken
#!{sys.executable} -m pip install torch
#!{sys.executable} -m pip install blobfile

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Initialize tiktoken tokenizer

In [3]:
from pathlib import Path
import tiktoken
from tiktoken.load import load_tiktoken_bpe
import torch
import json
import matplotlib.pyplot as plt

tokenizer_path = "./content/tokenizer.model"
num_reserved_special_tokens = 256

mergeable_ranks = load_tiktoken_bpe(tokenizer_path)

num_base_tokens = len(mergeable_ranks)
special_tokens = [
    "<|begin_of_text|>",
    "<|end_of_text|>",
    "<|reserved_special_token_0|>",
    "<|reserved_special_token_1|>",
    "<|finetune_right_pad_id|>",
    "<|step_id|>",
    "<|start_header_id|>",
    "<|end_header_id|>",
    "<|eom_id|>",
    "<|eot_id|>",
    "<|python_tag|>",
]
reserved_tokens = [
    f"<|reserved_special_token_{2 + i}|>"
    for i in range(num_reserved_special_tokens - len(special_tokens))
]
special_tokens = special_tokens + reserved_tokens

# source: https://github.com/meta-llama/llama-models/blob/main/models/llama3/api/tokenizer.py#L53
tokenizer = tiktoken.Encoding(
    name=Path(tokenizer_path).name,
    pat_str=r"(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+",
    mergeable_ranks=mergeable_ranks,
    special_tokens={token: len(mergeable_ranks) + i for i, token in enumerate(special_tokens)},
)

In [4]:
tokenizer.encode("hello")

[15339]

In [5]:
tokenizer.decode([15339])

'hello'

In [6]:
tokenizer.encode("hello Andrew")

[15339, 13929]

In [7]:
tokenizer.encode("hello andrew")

[15339, 323, 4361]

### Tokens.ipynb
If you would like to view a UTF-8 view of the Tokens.model file, uncomment the following line and run it.

In [8]:
#!cat Tokens.ipynb

You can also go to file->open to find Tokens.ipynb file. Please note that the file is large and opening it might take some time.

## Getting the length of tokens of an input text

In [9]:
input_text = "hello world"
len(tokenizer.encode(input_text))

2

In [10]:
question = "Who wrote the book Charlotte's Web?"
prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
len(encoded_tokens)

18

In [11]:
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]
for e, d in zip(encoded_tokens, decoded_tokens):
    print(e, d)

128000 <|begin_of_text|>
128006 <|start_header_id|>
882 user
128007 <|end_header_id|>
271 


15546 Who
6267  wrote
279  the
2363  book
29473  Charlotte
596 's
5000  Web
30 ?
128009 <|eot_id|>
128006 <|start_header_id|>
78191 assistant
128007 <|end_header_id|>
198 



In [12]:
from IPython.display import display, HTML
from utils import html_tokens, llama31

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>utils.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

In [13]:
display(HTML(html_tokens(decoded_tokens)))

In [14]:
#Try one of you own:
prompt = "Supercalifragilisticexpialidocious"
encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]
display(HTML(html_tokens(decoded_tokens)))

# LLM reasoning vs tokenization

In [15]:
question = "How many r's in the word strawberry?"
prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
response = llama31(prompt)
print(response)

There are 2 r's in the word "strawberry".


In [16]:
encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]
display(HTML(html_tokens(decoded_tokens)))

In [17]:
question = "How many r's in the word s t r a w b e r r y? "
prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
response = llama31(prompt)
print(response)

There are 3 r's in the word "strawberry".


In [18]:
encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]
display(HTML(html_tokens(decoded_tokens)))

# Extra examples

## Llama 3.1 tokenization model file demystification

The Llama 3.1 tokenization model, named as tokenizer.model, can be downloaded along with the Llama 3.1 model weights or from the Llama models repo.

In [19]:
# download the Llama 3.1 tokenizer model
#!wget https://raw.githubusercontent.com/meta-llama/llama-models/main/models/llama3/api/tokenizer.model

If you take a quick look at the model file, you'll see it has 128,000 lines and each line has two values separated by a space: a mysterious string and a number that starts with 0 and ends with 127,999.

In [20]:
!head -10 ./content/tokenizer.model

IQ== 0
Ig== 1
Iw== 2
JA== 3
JQ== 4
Jg== 5
Jw== 6
KA== 7
KQ== 8
Kg== 9


In [21]:
!tail -10 ./content/tokenizer.model

4LmM4LiB4Lij 127990
zrbOsQ== 127991
IOuNlOyasQ== 127992
2YjZhNin2Ko= 127993
0LLQsNGC0LjRgdGP 127994
IGvDtms= 127995
2YbYqA== 127996
INCy0YvRgdC+0LrQvtC5 127997
44O844O8 127998
6ZSm 127999


In [22]:
!wc -l ./content/tokenizer.model

  128000 ./content/tokenizer.model


Each line indeed describes one token out of 128K total tokens and its associated integer ID, and the string on each line is base64 encoded. Use the code snippet below to decode those 128K encoded strings, and then convert the decoded bytes to more readable UTF-8 tokens.

In [23]:
import base64

encoded_tokens = []
decoded_byte_tokens = []
decoded_utf8_tokens = []

with open("./content/tokenizer.model", 'r') as file:
  for i, line in enumerate(file):
    k, v = line.strip().split(' ')
    encoded_tokens.append({k: v})
    decoded_byte_tokens.append({base64.b64decode(k): v})
    decoded_utf8_tokens.append({base64.b64decode(k).decode('utf-8', errors="replace") : v})

Let's check the first ten encoded tokens (what's stored in the tokenizer.model), and their decoded byte and UTF-8 tokens.

In [24]:
list(encoded_tokens)[:10]

[{'IQ==': '0'},
 {'Ig==': '1'},
 {'Iw==': '2'},
 {'JA==': '3'},
 {'JQ==': '4'},
 {'Jg==': '5'},
 {'Jw==': '6'},
 {'KA==': '7'},
 {'KQ==': '8'},
 {'Kg==': '9'}]

In [25]:
list(decoded_byte_tokens)[:10]

[{b'!': '0'},
 {b'"': '1'},
 {b'#': '2'},
 {b'$': '3'},
 {b'%': '4'},
 {b'&': '5'},
 {b"'": '6'},
 {b'(': '7'},
 {b')': '8'},
 {b'*': '9'}]

In [26]:
list(decoded_utf8_tokens)[:10]

[{'!': '0'},
 {'"': '1'},
 {'#': '2'},
 {'$': '3'},
 {'%': '4'},
 {'&': '5'},
 {"'": '6'},
 {'(': '7'},
 {')': '8'},
 {'*': '9'}]

Let's confirm the tokenizer.model file stores the base64 encoded strings for tokens, e.g. the token "hello".

In [27]:
base64.b64encode('h'.encode('utf-8'))

b'aA=='

In [28]:
base64.b64encode('hello'.encode('utf-8'))

b'aGVsbG8='

In [29]:
!grep "aGVsbG8=" ./content/tokenizer.model

aGVsbG8= 15339


# More LLM reasoning vs tokenization

Let's try out Llama 3.1 on some recent tokenization related LLM problems, and see if we can improve its reasoning by some prompt engineering.

## Simple math problem

In [30]:
question = "Which number is bigger, 9.11 or 9.9? "
prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
response = llama31(prompt)
print(response)

9.11 is bigger than 9.9.


In [31]:
response = llama31(prompt, 70)
print(response)

9.9 is bigger than 9.11.


In [32]:
response = llama31(prompt, 405)
print(response)

9.11 is bigger than 9.9.

To compare these numbers, I looked at the decimal parts. Since 0.11 is greater than 0.09 (or 0.9), 9.11 is greater than 9.9.

Think of it like money: $9.11 is more than $9.90 (or $9.90 is 9 dollars and 90 cents, while $9.11 is 9 dollars and 11 cents, but since 11 cents is more than the difference between 90 cents and a dollar, it's actually more).


Somehow the largest Llama 3.1 405b model returns the incorrect result. From the visualization of the tokens in the prompt, you can see the number 9.11 is split into 3 tokens: "9", ".", and ".11", while 9.9 into 2 tokens: "9", ".", "9". If the two numbers are encoded as the two numbers themselves, correct model response will be more likely.

In [33]:
encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]
[x for x in zip(encoded_tokens, decoded_tokens)]

[(198, '\n'),
 (128000, '<|begin_of_text|>'),
 (128006, '<|start_header_id|>'),
 (882, 'user'),
 (128007, '<|end_header_id|>'),
 (271, '\n\n'),
 (23956, 'Which'),
 (1396, ' number'),
 (374, ' is'),
 (11493, ' bigger'),
 (11, ','),
 (220, ' '),
 (24, '9'),
 (13, '.'),
 (806, '11'),
 (477, ' or'),
 (220, ' '),
 (24, '9'),
 (13, '.'),
 (24, '9'),
 (30, '?'),
 (220, ' '),
 (128009, '<|eot_id|>'),
 (128006, '<|start_header_id|>'),
 (78191, 'assistant'),
 (128007, '<|end_header_id|>'),
 (198, '\n')]

In [34]:
display(HTML(html_tokens(decoded_tokens)))

## String reversing

First, for a common word "amazing", all 3 Llama 3.1 chat models reverse the string correctly.

In [35]:
input = "Reverse the string 'amazing'"
prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
response = llama31(prompt)
print(response)

The reverse of 'amazing' is 'gnizama'.


In [36]:
response = llama31(prompt, 70)
print(response)

The reverse of 'amazing' is 'gnizama'.


In [37]:
response = llama31(prompt, 405)
print(response)

The reverse of 'amazing' is 'gnizama'.


In [38]:
encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]
display(HTML(html_tokens(decoded_tokens)))

For a less common word "language", Llama 3.1 8B doesn't return the correct result, but 70B and 405B do.

In [39]:
input = "Reverse the string 'language'"
prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
response = llama31(prompt)
print(response)

The reverse of 'language' is 'egnualn'.


In [40]:
response = llama31(prompt, 70)
print(response)

egaugnal


In [41]:
response = llama31(prompt, 405)
print(response)

The reverse of 'language' is 'egaugnal'.


In [42]:
encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]
display(HTML(html_tokens(decoded_tokens)))

For the string "XMLElement", none of the 3 models is correct.

In [43]:
input = "Reverse the string 'XMLElement'"
prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
response = llama31(prompt)
print(response)

The reverse of 'XMLElement' is 'deMltxE'.


In [44]:
response = llama31(prompt, 70)
print(response)

'temnelMXE'


In [45]:
response = llama31(prompt, 405)
print(response)


The reverse of the string 'XMLElement' is 'TNEMELMX'.


In [46]:
encoded_tokens = tokenizer.encode(prompt, allowed_special="all")
decoded_tokens = [tokenizer.decode([token]) for token in encoded_tokens]
display(HTML(html_tokens(decoded_tokens)))